# Import Libraries and Set Options

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sqlite3
import collections

from daftpy.daftprep import (get_db, num_perc, process_price, process_coordinates, drop_coord_outliers, 
                             drop_floor_area, floor_area_wragling, process_floor_area, 
                             drop_info, process_info, process_views, process_rooms)

from daftpy.daftfeanalysis import missing_values

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')

# Load Data

We load the information through a fuction which drops the advertiser's personal information. 

In [3]:
sale = get_db(dbname='2021-11-25.db')

OperationalError: unable to open database file

In [ ]:
sale.info()

# Check Missing Values and Duplicates

In [ ]:
# Check missing values in absolute and relative terms
missing_values(sale)

The feature `energy_performance_indicator` has too many missing values, thus we should beware of them.

|Feature|% missing values|
|:---:|:---:|
|energy_performance_indicator|63|

As it has such a high percentage of missing values and it doesn't appear to be a critical feature we can drop it.

We shouldn't have any duplicated listing thanks to the *DuplicatesPipeline* implemented in the Scrapy project.

In [ ]:
sale.duplicated(keep=False).sum()

In [ ]:
sale.sample()

Next, let's start the cleansing and wrangling tasks. We will need cleaned prices and other features if we want to analize them.

# Energy Performance Indicator

We start dropping `energy_performance_indicator` column as we concluded above.

In [ ]:
print(f'Before dropping: {sale.shape}')
sale.drop(columns=['energy_performance_indicator'], inplace=True)
print(f'After dropping: {sale.shape}')

# Price

We have strange data points in the column. Let's investigate the different structures we can see:

In [ ]:
# sale['price'].value_counts()

In [ ]:
# The following are the diferent data point extructures that we can see in 
# the `price` column
sale['price'][[0, 8, 13, 28, 1442, 12842]]

**1. Price on Application**

`Price on Application` is the most repeated value in the `price` column. It equates to a 7.63% of ads.

In [ ]:
# Ads with `Price on Application` pattern
num_perc(df=sale, 
         feature='price', 
         pattern='Price on Application')

We will drop this kind of ads because we can't find out their prices.

**2. AMV: Price on Application**

In [ ]:
# Ads with `AMV: Price on Application` pattern
num_perc(df=sale, 
         feature='price', 
         pattern='AMV: Price on Application')

We will drop this kind of ads because we can't find out their prices.

**3. AMV: €725,000**

This patron is repeated a lot of times in the column, we will have to apply wrangling tasks on them.

**4. NaN**

In [ ]:
# Ads with missing values
num_perc(df=sale, 
         feature='price', 
         pattern=np.nan)

Advertisements with missing values in the `price` column are "Development" ads, not "Houses" ads. Development ads have several houses in it and that is why they have missing values. Furthermore, the houses belonging to developments are already scraped so we should drop the development ads.

In [ ]:
# You can check the above statement with the lines of code bellow

# sale[sale['price'].isna()] # Commented for sthetic reasons
sale.loc[sale['price'].isna(), 'url'].sample().values # url to check the ad

**5. £149,000 (€173,554)**

In [ ]:
# Ads with '£' pattern
num_perc(df=sale, 
         feature='price', 
         pattern='£')

Ads with prices in pounds are usually from North Ireland, which belongs to United Kingdom so we will drop these ads.

In [ ]:
# You can check the above statement with the lines of code bellow

# sale.dropna(subset=['price']).loc[
 #    sale.dropna(subset=['price'])['price'].str.contains('£')] # Commented for sthetic reasons

sale.dropna(subset=['price']).loc[
    sale.dropna(subset=['price'])['price'].str.contains('£'), 'url'
                                 ].sample().values # url to check the ad

**-> Conclusion**

Wrangling and cleansing tasks: 

|Value structure in ad's price|Number of ads|Percentage of ads|Task|
|:---:|:---:|:---:|:---:|
|€549,000|||Wrangling|
|AMV: €725,000|||Wrangling|
|£149,000 (€173,554)|25|0.16%|Drop|
|Price on Application|1174|7.63%|Drop|
|AMV: Price on Application|8|0.05%|Drop|
|NaN|278|1.81%|Drop|

Let's compare the before and after of transformations.

In [ ]:
pd.DataFrame({'before': sale['price'], 
              'after': process_price(sale)['price']}).head(10)

Let's do it!

In [ ]:
sale = process_price(sale.copy()).reset_index(drop=True)

# Coordinates

There are no missing values in the coordinates columns and the structure is the same in all data points. We will use the `process_coordinates` function to make one new column for `latitude` and another one for `longitude`. The function also drops the `coordinates` column once it has been processed.

In [ ]:
# Same structure in all data points in the column
sale['coordinates'].sample()

As the structure is the same in all ads, we can split the `coordinates` column in `+` using vectorized operations and create new columns from that. 

In [ ]:
sale = process_coordinates(sale)

We can use the new columns to make a scatter plot in order to check whether the points show a similar form to Ireland.

In [ ]:
# Check coordinates outliers 
sale.plot(kind='scatter', x='longitude', y='latitude', title='Houses for sale, Ireland');

As you can see above, there are some isolated points with a lower `latitude` that make it difficult to visualizate any country so the right thing would be drop them.

In [ ]:
# Drops coordinates outliers
sale = drop_coord_outliers(sale)

In [ ]:
sale.plot(kind='scatter', x='longitude', y='latitude', title='Houses for sale, Ireland'); # esta un poco achatada pero es Irlanda

# Floor Area

The most common pattern in the `floor_area` column follows a structure like this: `177 m²`. Nevertheless, there are several patterns which will need be handled.

In [ ]:
# sale['floor_area'].sample(30)

In [ ]:
# The following are the diferent data point extructures that we can see in 
# the `price` column
sale['floor_area'][[1861, 12076, 10284, 5114]]

**1. m²**

`floor_area` is a promising predictor but only a 55% of ads show this information. We will drop the other values but let's explore ads whose `floor_area` feature is mesured in `acres` to check whether we can keep them.

In [ ]:
# Ads with 'm²' pattern
num_perc(df=sale, 
         feature='floor_area', 
         pattern='m²')

**2. ac**

In [ ]:
# Ads with 'ac' pattern
num_perc(df=sale, 
         feature='floor_area', 
         pattern='ac')

If we check some ads we will see that ads with `ac` pattern refer to land extensions so we will drop them as well as the other ads with different patterns.

In [ ]:
sale.dropna(subset=['floor_area']).loc[
    sale.dropna(subset=['floor_area'])['floor_area'].str.contains('ac'), 'url'
                                 ].sample().values # url to check the ad

**3. NaN**

There are just two missing values becuse we dropped almost all of them when we dropped the ads with `NaN` values in the `price` column.

In [ ]:
# Ads with 'NaN' pattern
num_perc(df=sale, 
         feature='floor_area', 
         pattern=np.nan)

**-> Conclusions**

Wrangling and cleansing tasks: 

|Value structure in ad's price|Number of ads|Percentage of ads|Task|
|:---:|:---:|:---:|:---:|
|756 m²|7757|55.81%|Wrangling|
|2 ac|2540|18.27%|Drop|
|NaN|2|0.01%|Drop|
|For Sale by Private Treaty|||Drop|
|Section 27|||Drop|
|Thu, Nov 25th at 12:00 PM|||Drop|

Let's compare the before and after of transformations.

In [ ]:
pd.DataFrame({'before': sale.dropna(subset=['floor_area'])['floor_area'], 
              'after': process_floor_area(sale)['floor_area']}).head(10)

The following function helps us dropping all rows that do not contain the pattern `m²` as well as those with missing values. Furthermore, it will wrang the column to let us analyze the data in the next notebook.

In [ ]:
sale = process_floor_area(sale)

# Info

If we look at the info example we can see four variables into the same column, so it would be a good idea to split them in different columns.

In [ ]:
sale[['info', 'floor_area']].sample(5)

Also, we can see that there is no more missing values in the `info` column.

In [ ]:
sale['info'].isna().sum()

Let's check the possible data point's length after been splitted by a comma:

In [ ]:
# Create a DataFrame with the number of values with different lengths
pd.DataFrame({'Absolute': sale['info'].str.split(',').apply(len).value_counts(), 
              'Relative': sale['info'].str.split(',').apply(len).value_counts() / sale.shape[0]})

Let's dig a little deeper into the different cases.

**1. Length = 2**

Ads with splitted `info` length equal to two usually forget both the number of bedrooms and bathrooms.

In [ ]:
# Splitted `info` length = 2
#sale.dropna(subset=['info']).loc[sale['info'].dropna().str.split(',').apply(len) == 2, ['info', 'floor_area']].sample(5) 
sale.loc[sale['info'].str.split(',').apply(len) == 2, ['info', 'floor_area']].sample(5) 

**2. Length = 3**

If we check the rows which have a splitted `info` length of three we will see the following pattern where the advertiser forgot to add the number of bathrooms or the number of bedrooms. Usually he forgot the number of bathrooms.

In [ ]:
sale.loc[(sale['info'].str.split(',').apply(len) == 3), ['info', 'floor_area']].head()

**-> Conclusions**

Since rows that forget one or two values are so few we can drop them instead filling the lack of information.  

We will split the other ones into three columns as we already have the floor area.

Wrangling and cleansing tasks: 

|Value structure in ad's price|Number of ads|Percentage of ads|Task|
|:---:|:---:|:---:|:---:|
|4|7662|0.98|Wrangling: Split into 3 columns|
|3|81|0.01|Drop|
|2|14|0.001|Drop|

So let's do it! We will use a fuction to drop those rows which have a splitted `info` length less than four and we will split the other ones into three new columns as well as dropping the `info` column.

In [ ]:
sale = process_info(sale)

# Views

In [ ]:
sale[['views']].sample(3)

The only wrangling task we have to apply in the `views` column is to quit the comma. We have a simple function for that. 

In [ ]:
sale = process_views(sale)

# Bedroom and Bathroom

The only wrangling task we need to apply on `bedroom` and `bathroom` columns is isolating the number and erasing the word.

In [ ]:
sale[['bedroom']].sample(3)

In [ ]:
sale[['bathroom']].sample(3)

We have a simple function prepared to do it.

In [ ]:
sale = process_rooms(sale)

In [ ]:
sale[['bedroom', 'bathroom']].sample(3)

# Ber and Type House

![](imgs/ber.png)

[Building Energy Rating Certificate, SEAI](https://www.seai.ie/home-energy/building-energy-rating-ber/)

[Buiding Energy Rating Guide](https://www.seai.ie/publications/Your-Guide-to-Building-Energy-Rating.pdf)

`ber` and `type_house` columns don't need wrangling tasks since we extracted them pretty well thanks to Scrapy.

However, both columns contain aroun a 25% of missing values each one. We will leave them as they are for now.

In [ ]:
sale[['ber', 'type_house']].sample(3)

# PSR

The Register identifies licensed property service providers by name, number, licence type and location.

You can find more information about PSR Licence Number in the [Property Seervices Regulatory Authority website](http://www.psr.ie/en/PSRA/Pages/Register_Licensed_PSP).

In [ ]:
sale[['psr']].sample(3)

# Description

This information won't be used for now but I will keep it.

In [ ]:
sale['description'].sample().values

# Other columns

### Daft_id, Item_id, url, Name

These four columns don't seem very useful in order to infer houses prices. However, we should keep some of them for several reasons. 

- `daft_id`: This is the ad identifier provided by the website.
- `item_id`: After know that the website provides an identifier I bilt one so now it is completely useless. This identifier was extracted from the ad's url.
- `url`: The url could be useful as an indentifier but I prefer to give that task to the `daft.ie` column. The url will be saved just in case we need to check some advertisements.
- `name`: It can't be used as an idetifier because is possible to have some of them repeated. We will keep it because it could provide some information, but a priori I don't think we will use it.

|Column|What to do?|
|:---:|:---:|
|daft_ie|Keep|
|item_ie|Drop|
|url|Keep|
|name|Keep|

In [ ]:
sale.drop(columns='item_id', inplace=True)

### Sale Type

The `sale_type` column doesn't seem very interesting but we will keep it by the moment.

Also, there are a few ads with prices that seem to be from developments. We will consider those ads as a house.

In [ ]:
sale['sale_type'].value_counts()

In [ ]:
sale.loc[sale['sale_type']=='2 units available in this development', 'url'].values

### Entered-Renewed

`entered_renewed` is the date that the ad was entered or renewed. It doesn't need wrangling task but we could convert it to a datetime type. We will do that later in another notebook.

In [ ]:
sale[['entered_renewed']].sample(3)

### Type

This column shows whether the ad is for sale or for rent. There are houses and rooms for rent in the other table of the database.

In [ ]:
sale['type'].value_counts()

### scraping_date

In [ ]:
sale['scraping_date'].value_counts()

# Check Missing Values

In [ ]:
# Check missing values in absolute and relative terms
missing_values(sale)

# Save Data

In [ ]:
sale.info()

In [ ]:
sale.to_csv('data/sale_cleaned.csv', sep=',', index=False) 